In [2]:
from spinbox import *
from itertools import count
seeder = count(0,1) # generates a list of counting numbers for rng "seeds" (not really seeds in Numpy, but indices of seeds)

# The `spinbox` Tutorial Notebook

First, let me explain the purpose of this code and what it can do. In auxiliary field diffusion Monte Carlo (AFDMC) and Green's function Monte Carlo (GFMC) calculations of atomic nuclei, one approximates the many-body wavefunction with a linear combination of sample "states", which are propagated independently according to the nuclear Hamiltonian. These sample states are tensor product states in AFDMC and linear combinations of tensor product states in GFMC; since the latter is more general, any algorithm emplyed in AFDMC should be also applicable in GFMC in principle. This package provides the foundational elements for these calculations.

At the time of writing, `spinbox` is only concerned with spin-isospin degrees of freedom (i.e. SU(4)). There is an option to tack coordinates on to sample states, but it has not been developed further. One can optionally turn off isospin, so spin up/down is the only variable (i.e. SU(2)).

## Simple calculations in the full Hilbert space
For GFMC-style calculations one should use `HilbertState` and `HilbertOperator` classes, named as such because the entire basis of the Hilbert space is available. That is, each basis state is a tensor product state.

To start, let's instantiate some random GFMC-style states for a 2-particle system with just spin (no isospin) and compute stuff with them.

In [3]:
state_1 = HilbertState(n_particles=2, ketwise=True, isospin=False).randomize(seed=next(seeder))
print(state_1)

HilbertState ket of 2 particles:
[[ 0.06750061-0.2875841j ]
 [-0.07092296+0.19412905j]
 [ 0.34382285+0.70007675j]
 [ 0.05631758+0.50845808j]]


We have instatiated a random complex-values vector with 4 components corresponding to the states $|\uparrow\uparrow\rangle, |\uparrow\downarrow\rangle,|\downarrow\uparrow\rangle,|\downarrow\downarrow\rangle$.

Note that `ketwise=True` makes these ket states. Setting this to false makes bra states. Applying the `.dagger()` method conjugates this attribute.

We can instatiate another state and do some usual calculations.

In [4]:
print("< state 1 | state 1 > = " , state_1.dagger() * state_1)

state_2 = HilbertState(n_particles=2, ketwise=False, isospin=False).randomize(seed=next(seeder))

print("< state 2 | state 1 > = " , state_2 * state_1)

< state 1 | state 1 > =  [[1.+0.j]]
< state 2 | state 1 > =  [[0.12588545-0.23951561j]]


The `*` operator does most sensible multiplication. States are normalized by default.

You can multiply a state by a scalar number, but the scalar must go on the right, not the left. The reason for this has to do with Python's multiplication methods (I didn't want to define a `__rmul__` method for anything).

In [12]:
print(state_1 * 3.0) # reversing this order will give an error

HilbertState ket of 2 particles:
[[ 0.20250184-0.86275229j]
 [-0.21276888+0.58238715j]
 [ 1.03146854+2.10023025j]
 [ 0.16895275+1.52537425j]]


In [5]:
print("| state 1 > < state 2 | = " , state_1 * state_2)

| state 1 > < state 2 | =  HilbertOperator
Re=
[[ 0.13766273  0.08920391 -0.06410887  0.03841083]
 [-0.09717941 -0.07032561  0.03920967 -0.00989339]
 [-0.24990439 -0.01456018  0.23754086 -0.41483372]
 [-0.21393467 -0.08768083  0.14151299 -0.17899253]]
Im:
[[-0.01857179 -0.10003659 -0.06370059  0.20089113]
 [ 0.00139627  0.0620355   0.04960722 -0.14275895]
 [ 0.26845007  0.35358859  0.02266833 -0.34574636]
 [ 0.11000793  0.21491686  0.06685491 -0.30564766]]


Ah look, taking the outer product has produced a new class: a `HilbertOperator`. We can combine multiplications between states and operators. 

(Note that Python will associate multiplications to the left, not to the right as we often like to, so you may need to put in some parentheses.)



In [6]:
outer_product = state_1 * state_2
print("< state 1 | ( | state 1 > < state 2 | ) | state 2> = " , state_1.dagger() * outer_product * state_2.dagger())

< state 1 | ( | state 1 > < state 2 | ) | state 2> =  [[1.-4.16333634e-17j]]


We can make our own operator from scratch too. Operators are always instantiated to the identity. If you want to instantiate to zero, use the `.zero()` method. We can apply the Pauli spin operators to individual particles, and the isospin operators (if the isospin DOFs are included).

In [7]:
operator = HilbertOperator(n_particles=2, isospin=False).apply_sigma(particle_index=0, dimension=0)
print(operator)

HilbertOperator
Re=
[[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]
Im:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In the tensor product space formalism, this is the one-body operator $\hat{\sigma}_{x} \otimes I := \hat{\sigma}_{1x}$. Note that indices start at zero: particle $i$ is indexed $i-1$.

If we include isospin, we can apply $\tau$ operators too.

In [13]:
operator = HilbertOperator(n_particles=2, isospin=True).apply_sigma(particle_index=0, dimension=0).apply_tau(particle_index=1,dimension=2)
print(operator)

HilbertOperator
Re=
[[ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

In the tensor product space formalism, this is $\hat{\sigma}_{x} \otimes \hat{\tau}_z := \hat{\sigma}_{1x}\hat{\tau}_{2z}$. 

In the full Hilbert space, states and operators can be added together to produce states and operators respectively. This is not the case with `ProductState` and `ProductOperator`, which I will get to later.

Let's construct the two-body spin interaction operator for our two particles: $\vec{\sigma}_1 \cdot \vec{\sigma}_2$.

In [8]:
si_dot_sj = HilbertOperator(n_particles=2,isospin=False).zero()
for dimension in (0,1,2):
    si_dot_sj += HilbertOperator(n_particles=2,isospin=False).apply_sigma(0,dimension).apply_sigma(1,dimension)

print(si_dot_sj)

HilbertOperator
Re=
[[ 1.  0.  0.  0.]
 [ 0. -1.  2.  0.]
 [ 0.  2. -1.  0.]
 [ 0.  0.  0.  1.]]
Im:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


We can use the built-in `.exp()` method to exponentiate a `HilbertOperator` via Pade approximant. For instance, making an imaginary-time propagator object from $\vec{\sigma}_1 \cdot \vec{\sigma}_2$ might look like this:

In [16]:
propagator = si_dot_sj.scale(-0.1j).exp()
print(propagator)

HilbertOperator
Re=
[[0.99500417 0.         0.         0.        ]
 [0.         0.97517033 0.01983384 0.        ]
 [0.         0.01983384 0.97517033 0.        ]
 [0.         0.         0.         0.99500417]]
Im:
[[-0.09983342  0.          0.          0.        ]
 [ 0.          0.0978434  -0.19767681  0.        ]
 [ 0.         -0.19767681  0.0978434   0.        ]
 [ 0.          0.          0.         -0.09983342]]


However, `spinbox` has classes for propagators, so you probably want to use those instead.

## Simple calculations in the non-entangled space using product states

The classes `ProductState` and `ProductOperator` are analogous to `HilbertState` and `HilbertOperator` but they are individually restricted to tensor products of one-body states and one-body operators respectively.

In [17]:
state_1 = ProductState(n_particles=2, ketwise=True, isospin=False).randomize(seed=next(seeder))
print(state_1)

ProductState ket of 2 particles: 
ket #0:
[[ 0.08578222+0.8166101j ]
 [-0.23719504+0.51916074j]]
ket #1:
[[-0.15798429-0.12446437j]
 [-0.93378729+0.29595757j]]



Note that our usual matrix representation is replaced with just the one-body vectors. You can project a `ProductState` into a `HilbertState` by the `.to_manybody_basis()` method.

In [18]:
print(state_1.to_manybody_basis())

HilbertState ket of 2 particles:
[[ 0.08808662-0.1396884j ]
 [-0.32178429-0.73715223j]
 [ 0.1020901 -0.05249691j]
 [ 0.06784016-0.55498536j]]


Addition and subtraction are not allowed for `Product` objects, because in general doing so will not product another `Product` object. In other words, `Product` objects themselves do not constitute a formal vector space. 

We do, however, have scalar multiplication. There are two built-in ways to do it.

For a product state $|s_1\rangle \otimes |s_2\rangle$ multiplied by a scalar $c$,

$$c (|s_1\rangle \otimes |s_2\rangle) = c|s_1\rangle \otimes |s_2\rangle = |s_1\rangle \otimes c |s_2\rangle = c^{1/A}|s_1\rangle \otimes c^{1/A}|s_2\rangle$$

We can either choose a particle to multiply by $c$, or we multiply all of them by $c^{1/A}$.

In [20]:
print("Multiplying the first particle:")
print(state_1.scale_one(0,1000.0))
print("Multiplying the second particle:")
print(state_1.scale_one(1,1000.0))
print("Multiplying all particles:")
print(state_1.scale_all(1000.0))

Multiplying the first particle:
ProductState ket of 2 particles: 
ket #0:
[[  85.78222359+816.61010048j]
 [-237.19503594+519.16073507j]]
ket #1:
[[-0.15798429-0.12446437j]
 [-0.93378729+0.29595757j]]

Multiplying the second particle:
ProductState ket of 2 particles: 
ket #0:
[[ 0.08578222+0.8166101j ]
 [-0.23719504+0.51916074j]]
ket #1:
[[-157.98428809-124.46437366j]
 [-933.78728891+295.95757039j]]

Multiplying all particles:
ProductState ket of 2 particles: 
ket #0:
[[ 2.71267209+25.82347878j]
 [-7.50076563+16.41730395j]]
ket #1:
[[ -4.99590185-3.93590908j]
 [-29.52894683+9.35900013j]]



*Example*: Let's use the `ProductState` and `ProductOperator` classes to show $( \vec{\sigma}_1 \cdot \vec{\sigma}_2 + I)/2$ is equal to the spin exchange operator $P_{12}$.